Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [20]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels_raw = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels_raw = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels_raw = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000, 10)
Validation set (10000, 28, 28) (10000, 10)
Test set (10000, 28, 28) (10000, 10)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [21]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels_raw)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels_raw)
test_dataset, test_labels = reformat(test_dataset, test_labels_raw)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression with L2 Regularization

In [26]:
def logistic_regression_subsample(
    train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels,
    subsample_size=[50, 100, 1000, 5000, len(train_dataset)]):
    # Train logistic regression using 50, 100, 1000 and 5000 training examples
    # If we concatenate training and validation datasets after model selection 
    from sklearn.linear_model import LogisticRegression
    import time
    import random
    X_val = valid_dataset.reshape(-1, image_size * image_size)
    y_val = valid_labels_raw
    X_test = test_dataset.reshape(-1, image_size * image_size)
    y_test = test_labels_raw
    
    for k in subsample_size:
        start_time = time.time()
        print('Sample size: %i starts' % k)
        
        sample_index = random.sample(range(len(train_dataset)), k=k)
        X_train = train_dataset[sample_index].reshape(k, image_size * image_size)
        y_train = train_labels_raw[sample_index]
        
        logreg = LogisticRegression(solver='lbfgs', multi_class='multinomial')
        logreg.fit(X_train, y_train)

        accuracy_train = logreg.score(X_train, y_train)
        print('Training accuracy: %.4f' % accuracy_train)
        accuracy_val = logreg.score(X_val, y_val)
        print('Validation accuracy: %.4f' % accuracy_val)
        accuracy_test = logreg.score(X_test, y_test)
        print('Test accuracy: %.4f' % accuracy_test)
        print('Elapsed time: %.1f' % (time.time() - start_time))
        print('-----')

In [27]:
logistic_regression_subsample(\
    train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels)

Sample size: 50 starts
Training accuracy: 1.0000
Validation accuracy: 0.5790
Test accuracy: 0.6293
Elapsed time: 0.2
-----
Sample size: 100 starts
Training accuracy: 1.0000
Validation accuracy: 0.6994
Test accuracy: 0.7703
Elapsed time: 0.2
-----
Sample size: 1000 starts
Training accuracy: 1.0000
Validation accuracy: 0.7739
Test accuracy: 0.8505
Elapsed time: 0.3
-----
Sample size: 5000 starts
Training accuracy: 0.9722
Validation accuracy: 0.7718
Test accuracy: 0.8472
Elapsed time: 0.7
-----
Sample size: 200000 starts
Training accuracy: 0.8390
Validation accuracy: 0.8317
Test accuracy: 0.9006
Elapsed time: 32.6
-----


In [28]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
num_labels = 10
image_size = 28
batch_size = 128
num_hiddens1 = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hiddens1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hiddens1]))
    
    weights_output = tf.Variable(
        tf.truncated_normal([num_hiddens1 , num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    
    logits = tf.matmul(hidden1, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
        + beta * (tf.nn.l2_loss(weights_hidden1) + tf.nn.l2_loss(weights_output))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    
    valid_prediction = tf.nn.softmax(score_valid)
    
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    
    test_prediction = tf.nn.softmax(score_test)

In [29]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 643.658813
Minibatch accuracy: 10.2%
Validation accuracy: 28.4%
Minibatch loss at step 500: 195.602127
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 116.004028
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 68.427742
Minibatch accuracy: 92.2%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 41.213764
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.162882
Minibatch accuracy: 89.8%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.486997
Minibatch accuracy: 86.7%
Validation accuracy: 86.6%
Minibatch loss at step 3500: 9.567348
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 5.978190
Minibatch accuracy: 90.6%
Validation accuracy: 87.2%
Minibatch loss at step 4500: 3.767649
Minibatch accuracy: 87.5%
Validation accuracy: 87.1%
Minibatch loss at step 5000: 2.527497
Minibatch accuracy: 88.3%
Validati

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [32]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
num_labels = 10
image_size = 28
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_output = tf.Variable(
        tf.truncated_normal([image_size * image_size , num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_output) + biases_output
    
    logits = tf.matmul(tf_train_dataset, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    score_valid = tf.matmul(tf_valid_dataset, weights_output) + biases_output
    
    valid_prediction = tf.nn.softmax(score_valid)
    
    score_test = tf.matmul(tf_valid_dataset, weights_output) + biases_output
    
    test_prediction = tf.nn.softmax(score_test)

In [34]:
num_steps = 801

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step % 3) * batch_size
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.049930
Minibatch accuracy: 14.1%
Validation accuracy: 16.1%
Minibatch loss at step 100: 0.595023
Minibatch accuracy: 88.3%
Validation accuracy: 63.0%
Minibatch loss at step 200: 0.045626
Minibatch accuracy: 99.2%
Validation accuracy: 64.5%
Minibatch loss at step 300: 0.059171
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 400: 0.024278
Minibatch accuracy: 100.0%
Validation accuracy: 65.9%
Minibatch loss at step 500: 0.010654
Minibatch accuracy: 100.0%
Validation accuracy: 66.1%
Minibatch loss at step 600: 0.019320
Minibatch accuracy: 100.0%
Validation accuracy: 66.3%
Minibatch loss at step 700: 0.011535
Minibatch accuracy: 100.0%
Validation accuracy: 66.5%
Minibatch loss at step 800: 0.006140
Minibatch accuracy: 100.0%
Validation accuracy: 66.6%
Test accuracy: 10.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [35]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
num_labels = 10
image_size = 28
batch_size = 128
num_hiddens1 = 1024
beta = 0.001
num_steps = 10001
keep_prob = 0.3

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hiddens1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hiddens1]))
    
    weights_output = tf.Variable(
        tf.truncated_normal([num_hiddens1 , num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    
    # Add dropout
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
    
    logits = tf.matmul(hidden1, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    
    valid_prediction = tf.nn.softmax(score_valid)
    
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    
    test_prediction = tf.nn.softmax(score_test)

In [38]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 324.085205
Minibatch accuracy: 12.5%
Validation accuracy: 30.0%
Minibatch loss at step 500: 17.728594
Minibatch accuracy: 84.4%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 11.998902
Minibatch accuracy: 80.5%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 6.804618
Minibatch accuracy: 85.9%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 2.883880
Minibatch accuracy: 86.7%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 2.836493
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 1.711312
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 3500: 3.429536
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 4000: 3.956469
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 4500: 5.519013
Minibatch accuracy: 85.9%
Validation accuracy: 81.6%
Minibatch loss at step 5000: 3.567334
Minibatch accuracy: 86.7%
Validation acc

## Overfitting Scenario

In [39]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
num_labels = 10
image_size = 28
batch_size = 128
num_hiddens1 = 1024
beta = 0.001
num_steps = 10001
keep_prob = 0.3

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hiddens1]))
    biases_hidden1 = tf.Variable(tf.zeros([num_hiddens1]))
    
    weights_output = tf.Variable(
        tf.truncated_normal([num_hiddens1 , num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    score_hidden = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden)
    
    # Add dropout
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
    
    logits = tf.matmul(hidden1, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    score_hidden_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    score_valid = tf.matmul(
        tf.nn.relu(score_hidden_valid), weights_output) + biases_output
    
    valid_prediction = tf.nn.softmax(score_valid)
    
    score_hidden_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    score_test = tf.matmul(
        tf.nn.relu(score_hidden_test), weights_output) + biases_output
    
    test_prediction = tf.nn.softmax(score_test)

In [40]:
num_steps = 801

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step % 3) * batch_size
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 426.485138
Minibatch accuracy: 6.2%
Validation accuracy: 29.8%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 300: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 400: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 600: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 700: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Minibatch loss at step 800: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.3%
Test accuracy: 80.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [41]:
num_labels = 10
image_size = 28
batch_size = 128
num_hidden1 = 1024
num_hidden2 = 600
num_hidden3 = 200
sd_hidden1 = np.sqrt(2.0/1000)
sd_hidden2 = np.sqrt(2.0/num_hidden1)
sd_hidden3 = np.sqrt(2.0/num_hidden2)
sd_output = np.sqrt(2.0/num_hidden3)
keep_prob1 = 0.95
keep_prob2 = 0.6
keep_prob3 = 0.7
beta1 = 0.0001
beta2 = 0.0001
beta3 = 0.0001
beta4 = 0.0001
num_steps = 20001
start_learning_rate = 0.4
decay_steps = 1000
decay_rate = 0.96


In [42]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden1], stddev=sd_hidden1))
    biases_hidden1 = tf.Variable(tf.zeros([num_hidden1]))
    weights_hidden2 = tf.Variable(
        tf.truncated_normal([num_hidden1, num_hidden2], stddev=sd_hidden2))
    biases_hidden2 = tf.Variable(tf.zeros([num_hidden2]))
    weights_hidden3 = tf.Variable(
        tf.truncated_normal([num_hidden2, num_hidden3], stddev=sd_hidden3))
    biases_hidden3 = tf.Variable(tf.zeros([num_hidden3]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden3, num_labels], stddev=sd_output))
    biases_output = tf.Variable(tf.zeros([num_labels]))
  
    # Computate training.
    score_hidden1 = tf.matmul(tf_train_dataset, weights_hidden1) + biases_hidden1
    hidden1 = tf.nn.relu(score_hidden1)
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob1)
    score_hidden2 = tf.matmul(hidden1_drop, weights_hidden2) + biases_hidden2
    hidden2 = tf.nn.relu(score_hidden2)
    hidden2_drop = tf.nn.dropout(hidden2, keep_prob2)
    score_hidden3 = tf.matmul(hidden2_drop, weights_hidden3) + biases_hidden3
    hidden3 = tf.nn.relu(score_hidden3)
    hidden3_drop = tf.nn.dropout(hidden3, keep_prob3)
    
    score = tf.matmul(hidden3_drop, weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=score, labels=tf_train_labels))\
        + (beta1 * tf.nn.l2_loss(weights_hidden1)
           + beta2 * tf.nn.l2_loss(weights_hidden2)
           + beta3 * tf.nn.l2_loss(weights_hidden3)
           + beta4 * tf.nn.l2_loss(weights_output))
    
    # Add learning rate decay
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(
        start_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
  
    # Optimizer.
    optimizer = (tf.train.GradientDescentOptimizer(learning_rate)
                   .minimize(loss, global_step=global_step))
  
    # Predictions for the training, validation, and test data.    
    train_prediction = tf.nn.softmax(score)
    
    score_hidden1_valid = tf.matmul(tf_valid_dataset, weights_hidden1) + biases_hidden1
    hidden1_valid = tf.nn.relu(score_hidden1_valid)
    score_hidden2_valid = tf.matmul(hidden1_valid, weights_hidden2) + biases_hidden2
    hidden2_valid = tf.nn.relu(score_hidden2_valid)
    score_hidden3_valid = tf.matmul(hidden2_valid, weights_hidden3) + biases_hidden3
    hidden3_valid = tf.nn.relu(score_hidden3_valid)
    score_valid = tf.matmul(hidden3_valid, weights_output) + biases_output    
    valid_prediction = tf.nn.softmax(score_valid)
    
    score_hidden1_test = tf.matmul(tf_test_dataset, weights_hidden1) + biases_hidden1
    hidden1_test = tf.nn.relu(score_hidden1_test)
    score_hidden2_test = tf.matmul(hidden1_test, weights_hidden2) + biases_hidden2
    hidden2_test = tf.nn.relu(score_hidden2_test)
    score_hidden3_test = tf.matmul(hidden2_test, weights_hidden3) + biases_hidden3
    hidden3_test = tf.nn.relu(score_hidden3_test)
    score_test = tf.matmul(hidden3_test, weights_output) + biases_output    
    test_prediction = tf.nn.softmax(score_test)

In [44]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.535981
Minibatch accuracy: 8.6%
Validation accuracy: 32.3%
Minibatch loss at step 500: 0.529990
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 0.567146
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Minibatch loss at step 1500: 0.367720
Minibatch accuracy: 93.8%
Validation accuracy: 87.5%
Minibatch loss at step 2000: 0.405763
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 0.481014
Minibatch accuracy: 88.3%
Validation accuracy: 88.2%
Minibatch loss at step 3000: 0.506625
Minibatch accuracy: 86.7%
Validation accuracy: 88.5%
Minibatch loss at step 3500: 0.591470
Minibatch accuracy: 86.7%
Validation accuracy: 88.8%
Minibatch loss at step 4000: 0.441176
Minibatch accuracy: 93.0%
Validation accuracy: 89.0%
Minibatch loss at step 4500: 0.352952
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.533545
Minibatch accuracy: 90.6%
Validation accuracy